Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:

- GaussianNB.
- LogisticRegression.
- Linear + PolynomialFeatures.
- RandomForestClassifier.
При обучении использовать make_pipeline
GridSearchCV для поиска параметров 
Посчитать матрики (f1 + точность + AUC). 

In [15]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV

Загрузим предварительно сохраненные подготовленные данные

In [16]:
result = pd.read_csv('ML.csv')

In [17]:
result = result.set_index('client_id')

In [18]:
result

,age,credit_month,monthly_income,credit_sum,credit_count,score_shk,gender_F,gender_M,marital_status_CIV,marital_status_DIV,...,living_region_УЛЬЯНОВСКАЯ,living_region_ХАБАРОВСКИЙ,living_region_ХАНТЫМАНСИЙСКИЙ,living_region_ХАНТЫМАНСИЙСКИЙ,living_region_ЧЕЛЯБИНСКАЯ,living_region_ЯКУТИЯ,living_region_ЯМАЛОНЕНЕЦКИЙ,living_region_ЯРОСЛАВСКАЯ,tariff_id,open_account_flg
client_id,,,,,,,,,,,,,,,,,,,,,
8,-0.988118,-0.620867,1.558927,2.267526,0.773954,0.334511,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01,False
13,0.084609,-0.620867,2.202081,1.759238,-0.618431,1.825065,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-01,False
16,-0.207953,-0.620867,-0.113271,-0.885423,0.077762,0.215738,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-01,False
17,-0.012912,-0.620867,0.915774,-0.030274,1.470147,0.089450,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-04-01,False
18,1.742458,1.254834,0.272621,0.185992,2.862532,-0.489213,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024-01-01,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170740,-0.402994,-0.620867,-0.692109,-0.696895,0.077762,0.353537,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01,False
170741,-1.183159,1.254834,-0.885055,-0.284372,-0.618431,1.265595,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-01,False
170744,-0.500515,-0.620867,0.272621,0.419398,-0.618431,-0.468072,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2032-01-01,False


В датасете содержиться неодинаковое количество true и false в open_account_flg. Выполним выравнивание данных

In [19]:
false_class = result[result['open_account_flg'] == False]
true_class = result[result['open_account_flg'] == True]

min_class_size = min(len(false_class), len(true_class))

# Уменьшаем оба класса до одинакового размера

balanced_false = false_class.sample(n=min_class_size, random_state=42)
balanced_true = true_class.sample(n=min_class_size, random_state=42)

# Объединяем обратно
balanced_df = pd.concat([balanced_false, balanced_true]).sample(frac=1, random_state=42)

# Выводим размеры классов для проверки
print(balanced_df['open_account_flg'].value_counts())

# Датафрейм balanced_df содержит одинаковое количество true и false в поле open_account_flg.

open_account_flg
False    20358
True     20358
Name: count, dtype: int64


In [20]:
X = balanced_df.copy()
del X['open_account_flg'] # X - данные (удаляем поле open_account_flg)

Y = balanced_df['open_account_flg'] # Y - ключи (поле open_account_flg)

In [21]:
# Разделим на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

# Разделение на train и test
X_train, X_test, y_train, y_test = train_test_split(
    X,Y,
    test_size=0.5,      # Половина данных в тестовую выборку
    random_state=42,    # Фиксация случайного разбиения для воспроизводимости
   # shuffle=True        # Перемешивание перед разделением
)

print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)

Размер обучающей выборки: (20358, 92)
Размер тестовой выборки: (20358, 92)


In [22]:
# Удалим поле с датами

del X_train['tariff_id'] 
del X_test['tariff_id'] 

In [23]:
def perform_grid_search(model, param_grid, X_train = X_train, y_train = y_train, X_test=X_test, y_test=y_test, scoring='f1', cv=2, verbose=0,threshold = None):
  
    # Настройка GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=scoring,
        cv=cv,
        verbose=verbose,
        n_jobs=-1
    )
    
    # Запуск поиска по сетке
    grid_search.fit(X_train, y_train)
    
    # Результаты поиска
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    
    print("Лучшие параметры:", best_params)
    
    
    if X_test is not None and y_test is not None:
        y_pred = best_model.predict(X_test)
        
        if threshold: 
            y_pred_proba = y_pred
            y_pred = (y_pred > threshold).astype(int) # Преобразуем вероятности в классы
        else:
            y_pred_proba = best_model.predict_proba(X_test)[:, 1] # Получаем вероятности используя метод модели

        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba)
        }
        
        print("\nРезультаты на тестовой выборке:")
        print(f"Accuracy: {metrics['accuracy']}")
        print(f"F1-score: {metrics['f1']}")
        print(f"ROC AUC: {metrics['roc_auc']}")

GaussianNB

In [24]:
param_grid_gnb = {
    'var_smoothing': [1e-11, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-3, 0.01, 0]  # Параметры сглаживания
}

perform_grid_search(GaussianNB(), param_grid_gnb)

Лучшие параметры: {'var_smoothing': 0.01}

Результаты на тестовой выборке:
Accuracy: 0.5671480499066706
F1-score: 0.5190481388494705
ROC AUC: 0.5941857579334855


LogisticRegression

In [29]:
param_grid_logreg = [
    # l1 регуляризация
    {
        'penalty': ['l1'],
        'solver': ['liblinear', 'saga'],
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [100, 200, 500]
    },
    # l2 регуляризация
    {
        'penalty': ['l2'],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'saga'],
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [100, 200, 500]
    },
    # elasticnet регуляризация
    {
        'penalty': ['elasticnet'],
        'solver': ['saga'],
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [100, 200, 500],
        'l1_ratio': [0.1, 0.5, 0.9]  # Только для elasticnet
    },
    # Без регуляризации
    {
        'penalty': [None],
        'solver': ['lbfgs', 'newton-cg', 'saga'],
        'max_iter': [100, 200, 500]
    }
]
perform_grid_search(LogisticRegression(), param_grid_logreg)

Лучшие параметры: {'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'elasticnet', 'solver': 'saga'}

Результаты на тестовой выборке:
Accuracy: 0.5951959917477159
F1-score: 0.5987242537858499
ROC AUC: 0.6358554457367573


LinearRegression+PolynomialFeatures

In [30]:
param_grid_poly_lr = {
    'poly__degree': [1, 2, 3, 4]  # Степень полинома
}
poly_lin_model = Pipeline([
    ('poly', PolynomialFeatures()),
    ('linear', LinearRegression())
])

perform_grid_search(poly_lin_model, param_grid_poly_lr, scoring='neg_mean_squared_error', threshold=0.5)

Лучшие параметры: {'poly__degree': 1}

Результаты на тестовой выборке:
Accuracy: 0.5932802829354553
F1-score: 0.5959398789771618
ROC AUC: 0.6344322539169375


RandomForestClassifier

In [27]:
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

perform_grid_search(RandomForestClassifier(), param_grid_rf)

Лучшие параметры: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}

Результаты на тестовой выборке:
Accuracy: 0.6151881324295118
F1-score: 0.6067269076305221
ROC AUC: 0.6710240191749706


В результате выполнения задания построены модели GaussianNB, LogisticRegression, LinearRegression+PolynomialFeatures, RandomForestClassifier и выполнен подбор их параметров.

Лучший результат получен при использовании модели RandomForestClassifier (F1-score: 0.6067269076305221).
Худший результат дала модель GaussianNB (F1-score: 0.5190481388494705).